# Computation

## General concepts and mechanisms

### Overview

Binary operations between data arrays or datasets behave as follows:

| Property | Action |
| --- | --- |
| coord | compare, abort on mismatch |
| data | apply operation |
| mask | combine with `or` |
| attr | typically ignored or dropped |

In the special case of in-place operations such as `+=` or `*=` scipp preserves existing attributes and ignores attributes of the right-hand-side.

### Dimension matching and transposing

Operations "align" variables based on their dimension labels.
That is, an operation between two variables that have a transposed memory layout behave correctly:

In [ ]:
import numpy as np
import scipp as sc

a = sc.Variable(values=np.random.rand(2, 4),
                variances=np.random.rand(2, 4),
                dims=['x', 'y'],
                unit=sc.units.m)
b = sc.Variable(values=np.random.rand(4, 2),
                variances=np.random.rand(4, 2),
                dims=['y', 'x'],
                unit=sc.units.s)
a/b

### Propagation of uncertainties

If variables have variances, operations correctly propagate uncertainties (the variances), in contrast to a naive implementation using numpy:

In [ ]:
result = a/b
result.values

In [ ]:
a.values/np.transpose(b.values)

In [ ]:
result.variances

In [ ]:
a.variances/np.transpose(b.variances)

The implementation assumes uncorrelated data and is otherwise based on, e.g., [Wikipedia: Propagation of uncertainty](https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Example_formulae>).
See also [Propagation of uncertainties](../python-reference/error-propagation.rst) for the concrete equations used for error propagation.

### Broadcasting

Missing dimensions in the operands are automatically broadcast.
Consider:

In [ ]:
var_xy = sc.Variable(dims=['x', 'y'], values=np.arange(6).reshape((2,3)))
print(var_xy.values)

In [ ]:
var_y = sc.Variable(dims=['y'], values=np.arange(3))
print(var_y.values)

In [ ]:
var_xy -= var_y
print(var_xy.values)

Since `var_y` did not depend on dimension `'x'` it is considered as "constant" along that dimension.
That is, the *same* `var_y` values are subtracted *from all slices of dimension `'x'`* in `var_xy`.

Coming back to our original variables `a` and `b`, we see that broadcasting integrates seamlessly with slicing and transposing:

In [ ]:
a.values

In [ ]:
a -= a['x', 1]
a.values

Both operands may be broadcast, creating an output with the combination of input dimensions:

In [ ]:
sc.show(a['x', 1])
sc.show(a['y', 1])
sc.show(a['x', 1] + a['y', 1])

Note that in-place operations such as `+=` will never change the shape of the left-hand-side.
That is only the right-hand-side operation can be broadcast, and the operation fails of a broadcast of the left-hand-side would be required.

### Units

Units are required to be compatible:

In [ ]:
try:
    a + b
except Exception as e:
    print(str(e))

### Coordinate and name matching

In operations with datasets, data items are paired based on their names when applying operations to datasets.
Operations fail if names do not match:

- In-place operations such as `+=` accept a right-hand-side operand that omits items that the left-hand-side has.
  If the right-hand-side contains items that are not in the left-hand-side the operation fails.
- Non-in-place operations such as `+` return a new dataset with items from the intersection of the inputs.

Coords are compared in operations with datasets or data arrays (or items of datasets).
Operations fail if there is any mismatch in coord or label values.

In [ ]:
d1 = sc.Dataset(
    {'a': sc.Variable(dims=['x', 'y'], values=np.random.rand(2, 3)),
     'b': sc.Variable(dims=['y', 'x'], values=np.random.rand(3, 2)),
     'c': sc.Variable(dims=['x'], values=np.random.rand(2)),
     'd': sc.Variable(1.0)},
    coords={
        'x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(3.0), unit=sc.units.m)})
d2 = sc.Dataset(
    {'a': sc.Variable(dims=['x', 'y'], values=np.random.rand(2, 3)),
     'b': sc.Variable(dims=['y', 'x'], values=np.random.rand(3, 2))},
    coords={
        'x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(3.0), unit=sc.units.m)})

In [ ]:
d1 += d2

In [ ]:
try:
    d2 += d1
except Exception as e:
    print(str(e))

In [ ]:
d3 = d1 + d2
for name in d3:
    print(name)

In [ ]:
d3['a'] -= d3['b'] # transposing
d3['a'] -= d3['x', 1]['b'] # broadcasting
try:
    d3['a'] -= d3['x', 1:2]['b'] # fail due to coordinate mismatch
except Exception as e:
    print(str(e))

## Arithmetics

The arithmetic operations `+`, `-`, `*`, and `/` and their in-place variants `+=`, `-=`, `*=`, and `/=` are available for variables, data arrays, and datasets.
They can also be combined with [slicing](slicing.ipynb).

## Trigonometrics

Trigonometric functions like `sin` are supported for variables.
Units for angles provide a safeguard and ensure correct operation when working with either degree or radian:

In [ ]:
rad = 3.141593*sc.units.rad
deg = 180.0*sc.units.deg
print(sc.sin(rad))
print(sc.sin(deg))
try:
    rad + deg
except Exception as e:
    print(str(e))

## Other

See the [list of free functions](../python-reference/api.rst#free-functions) for an overview of other available operations.